<a href="https://colab.research.google.com/github/Loumstar/Improving-RoBERTa/blob/master/main_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Thu Mar  3 13:33:26 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P8    32W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Setup


## Installations & Imports

In [2]:
!pip install transformers sentencepiece ipywidgets tensorboardx wandb
!jupyter nbextension enable --py widgetsnbextension

     |████████████████████████████████| 3.5 MB 11.2 MB/s 
     |████████████████████████████████| 1.2 MB 39.0 MB/s 
     |████████████████████████████████| 125 kB 51.5 MB/s 
     |████████████████████████████████| 1.7 MB 25.4 MB/s 
     |████████████████████████████████| 895 kB 43.4 MB/s 
     |████████████████████████████████| 6.5 MB 34.6 MB/s 
     |████████████████████████████████| 67 kB 4.8 MB/s 
     |████████████████████████████████| 596 kB 45.8 MB/s 
     |████████████████████████████████| 144 kB 47.5 MB/s 
     |████████████████████████████████| 181 kB 39.7 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=1bd22277fd79e0e74dcbf1df90f35ca8efce354c2d4feefd8189832938c2ea58
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13

In [3]:
import logging
import string
import wandb
import nltk
import torch

import torch.nn as nn
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
from urllib import request

from transformers import Trainer, TrainingArguments
from transformers import DebertaTokenizer, DebertaForSequenceClassification

from sklearn.utils import shuffle
from sklearn.metrics import classification_report, f1_score

nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('punkt')

from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
from nltk.stem import WordNetLemmatizer

from torch.utils.data import DataLoader

if not torch.cuda.is_available():
  print('WARNING: You may want to change the runtime to GPU for faster training!')
  DEVICE = 'cpu'
else:
  print("CUDA available.")
  DEVICE = 'cuda:0'

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
CUDA available.


## Mount Google Drive

In [4]:
from google.colab import drive
drive.mount('/content/drive')

%cd '/content/drive/MyDrive/'
%mkdir './nlp-cswrk'
%cd './nlp-cswrk' 

Mounted at /content/drive
/content/drive/MyDrive
mkdir: cannot create directory ‘./nlp-cswrk’: File exists
/content/drive/MyDrive/nlp-cswrk


## Logging

In [5]:
# prepare logger
logging.basicConfig(level=logging.INFO)

transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

## Download Datasets

In [6]:
module_url = f"https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/practice%20splits/train_semeval_parids-labels.csv"
module_name = module_url.split('/')[-1]

print(f'Fetching {module_url}')
with request.urlopen(module_url) as f, open(module_name,'w') as outf:
  b = f.read()
  outf.write(b.decode('utf-8'))

Fetching https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/practice%20splits/train_semeval_parids-labels.csv


In [7]:
module_url = f"https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/practice%20splits/dev_semeval_parids-labels.csv"
module_name = module_url.split('/')[-1]

print(f'Fetching {module_url}')
with request.urlopen(module_url) as f, open(module_name,'w') as outf:
  b = f.read()
  outf.write(b.decode('utf-8'))

Fetching https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/practice%20splits/dev_semeval_parids-labels.csv


In [8]:
module_url = f"https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/dont_patronize_me.py"
module_name = module_url.split('/')[-1]

print(f'Fetching {module_url}')
with request.urlopen(module_url) as f, open(module_name,'w') as outf:
  a = f.read()
  outf.write(a.decode('utf-8'))

Fetching https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/dont_patronize_me.py


In [9]:
from dont_patronize_me import DontPatronizeMe

dpm = DontPatronizeMe('.', '.')
dpm.load_task1()

In [10]:
# helper function to save predictions to an output file
def labels2file(p, outf_path):
	with open(outf_path,'w') as outf:
		for pi in p:
			outf.write(','.join([str(k) for k in pi])+'\n')

# Pre-processing

## Data Formatting

### Convert CSVs to Data Frames

In [11]:
train_import = pd.read_csv('train_semeval_parids-labels.csv')
test_import = pd.read_csv('dev_semeval_parids-labels.csv')

In [12]:
train_import.par_id = train_import.par_id.astype(str)
test_import.par_id = test_import.par_id.astype(str)

In [13]:
rows = [] # will contain par_id, label and text
for idx in range(len(train_import)):
  parid = train_import.par_id[idx]
  #print(parid)
  # select row from original dataset to retrieve `text` and binary label
  text = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].text.values[0]
  label = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].label.values[0]
  keyword = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].keyword.values[0]
  country = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].country.values[0]
  intensity = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].orig_label.values[0]
  length = len(dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].text.values[0])
  rows.append({
      'par_id':parid,
      'text':text,
      'label':label,
      'keyword':keyword,
      'country':country,
      'intensity':intensity,
      'length':length
  })
train_df = pd.DataFrame(rows)

In [14]:
rows = [] # will contain par_id, label and text
for idx in range(len(test_import)):  
  parid = test_import.par_id[idx]
  #print(parid)
  # select row from original dataset
  text = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].text.values[0]
  label = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].label.values[0]
  keyword = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].keyword.values[0]
  country = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].country.values[0]
  intensity = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].orig_label.values[0]
  length = len(dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].text.values[0])
  rows.append({
      'par_id':parid,
      'text':text,
      'label':label,
      'keyword':keyword,
      'country':country,
      'intensity':intensity,
      'length':length
  })
  test_df = pd.DataFrame(rows)

### Shuffle Datasets

In [15]:
train_df = shuffle(train_df)
test_df = shuffle(test_df)

## Implementations

### Main Function
Enter iteration number to pre-process data corresponding to the implementation iteration.

In [16]:
def main_implement(implementation_number, train):
    if implementation_number == 1:
        train = downsample_unpat(train)
    elif implementation_number == 2:
        train = downsample_unpat(train)
        train = upsample_pat(train)
    elif implementation_number == 3:
        train = downsample_unpat(train)
        train = lemmatise_df(train)
    elif implementation_number == 4:
        train = add_backtranslations(train)
        train = downsample_unpat(train)
    
    return train

In [17]:
def get_label_ratio(dataset):
    return len(dataset[dataset.label == 0])/len(dataset[dataset.label == 1])

### Implementation 1
Downsample unpatronising data (as in competition's roberta base) (ratio from 9:1 to 3:1).

In [18]:
def downsample_unpat(train):
    pat_train = train[train.label == 1]
    npos = len(pat_train)

    train = pd.concat([pat_train,train[train.label == 0][:npos*3]])
    return train

### Implementation 2
Upsample patronising data

In [19]:
def upsample_pat(train):
    ratio = int(get_label_ratio(train))

    pat_train = train[train.label == 1]
    unpat_train = train[train.label == 0]

    pat_train = pd.concat([pat_train]*ratio)
    train = pd.concat([pat_train, unpat_train])
    
    return train

In [20]:
def df_to_dict_of_lists(df):
    texts = df['text'].values.tolist()
    labels = df['label'].values.tolist()
    parids = df['par_id'].values.tolist()
    keywords = df['keyword'].values.tolist()
    countries = df['country'].values.tolist()
    intensities = df['intensity'].values.tolist()
    lengths = df['length'].values.tolist()
    
    return {
        'texts': texts,
        'labels': labels,
        'parids': parids,
        'keywords': keywords,
        'countries': countries,
        'intensities': intensities,
        'lengths': lengths
    }

### Implementation 3
Lemmatise words in text using verb PoS tagging

In [21]:
lemmatizer = WordNetLemmatizer()

def remove_punctuation(token):
    return not any(map(lambda s: s in string.punctuation, token))

def lemmatise(word):
    return lemmatizer.lemmatize(word, pos="v")

def lemmatise_sentence(sentence):
  tokens = wordpunct_tokenize(sentence)
  tokens = list(map(lemmatise, tokens))
  tokens = list(filter(remove_punctuation, tokens))

  return " ".join(tokens)

def lemmatise_df(df):
  df.text = df.text.map(lemmatise_sentence)
  
  return df

### Implementation 4
Generate new examples using back-translation (of French and Spanish)



In [22]:
module_url = f"https://raw.githubusercontent.com/Loumstar/Improving-RoBERTa/master/data/dontpatronizeme_french_backtranslation.csv" 
module_name = module_url.split('/')[-1]

print(f'Fetching {module_url}')
with request.urlopen(module_url) as f, open(module_name,'w') as outf:
  b = f.read()
  outf.write(b.decode('latin-1'))

Fetching https://raw.githubusercontent.com/Loumstar/Improving-RoBERTa/master/data/dontpatronizeme_french_backtranslation.csv


In [23]:
module_url = f"https://raw.githubusercontent.com/Loumstar/Improving-RoBERTa/master/data/dontpatronizeme_spanish_backtranslation.csv" 
module_name = module_url.split('/')[-1]

print(f'Fetching {module_url}')
with request.urlopen(module_url) as f, open(module_name,'w') as outf:
  b = f.read()
  outf.write(b.decode('latin-1'))

Fetching https://raw.githubusercontent.com/Loumstar/Improving-RoBERTa/master/data/dontpatronizeme_spanish_backtranslation.csv


In [24]:
backtranslated_cols = ["par_id", "art_id", "category", "country", "text", "score"]

french_import = pd.read_csv('dontpatronizeme_french_backtranslation.csv', names=backtranslated_cols)
spanish_import = pd.read_csv('dontpatronizeme_spanish_backtranslation.csv', names=backtranslated_cols)

In [25]:
def get_list_length(x):
  if type(x) == list:
    return len(x)
  elif type(x) != None:
    return 1
  else:
    return None

# Copy the training dataset and swap out the text 
# for the back-translated version.
french_df = train_df.copy()
french_df.text = french_import.text

spanish_df = train_df.copy()
spanish_df.text = spanish_import.text

def add_backtranslations(df):
  df = df.append(french_df, ignore_index=True, 
                 verify_integrity=True)
  
  df = df.append(spanish_df, ignore_index=True, 
                 verify_integrity=True)
  
  df.length = df.text.str.split().map(get_list_length)
  df.dropna(inplace=True)
  
  return df

In [26]:
print(train_df.head())

print(french_df.head())
print(spanish_df.head())

     par_id  ... length
1216    466  ...    325
7962   7930  ...    234
7920   7883  ...     74
7876   7834  ...     62
3233   2708  ...    365

[5 rows x 7 columns]
     par_id  ... length
1216    466  ...    325
7962   7930  ...    234
7920   7883  ...     74
7876   7834  ...     62
3233   2708  ...    365

[5 rows x 7 columns]
     par_id  ... length
1216    466  ...    325
7962   7930  ...    234
7920   7883  ...     74
7876   7834  ...     62
3233   2708  ...    365

[5 rows x 7 columns]


### Choose Implementation

In [27]:
train_df = main_implement(3, train_df)

In [28]:
print(get_label_ratio(train_df))

3.0


## More Data Formatting

Convert data to class and create embeddings from tokeniser

In [29]:
class PatroDataset(torch.utils.data.Dataset):

    def __init__(self, tokenizer, input_set, max_length=128):

        self.tokenizer = tokenizer
        self.texts = input_set['texts']
        self.labels = input_set['labels']
        self.parids = input_set['parids']
        self.keywords = input_set['keywords']
        self.countries = input_set['countries']
        self.intensities = input_set['intensities']
        self.lengths = input_set['lengths']

        self.max_length = max_length

    def collate_fn(self, batch):
        texts, labels = [], []

        for b in batch:
            texts.append(b['text'])
            labels.append(b['label'])

        encodings = self.tokenizer(texts, return_tensors='pt', 
                                   padding=True, truncation=True, 
                                   max_length=self.max_length)
        
        encodings['label'] =  torch.tensor(labels)
        
        return encodings
    
    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
       
        return {
            'text': self.texts[idx],
            'label': self.labels[idx],
            'parid': self.parids[idx],
            'keyword': self.keywords[idx],
            'countriy': self.countries[idx],
            'intensity': self.intensities[idx],
            'length': self.lengths[idx]
        }

In [30]:
tokenizer = DebertaTokenizer.from_pretrained("microsoft/deberta-base")

Downloading:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/474 [00:00<?, ?B/s]

In [31]:
trainset = df_to_dict_of_lists(train_df)
testset = df_to_dict_of_lists(test_df)

test_dataset = PatroDataset(tokenizer, testset)

## Model

##### Hyperparameter Tuning

In [32]:
hyperparameter_tuning = False

if hyperparameter_tuning:
    wandb.login()

    config = {
        "name": "vanilla-sweep",
        "method": "bayes",
        "metric": {
            "name": "f1_score",
            "goal": "maximize"
        },
        "parameters": {
            "num_epochs": {
                "min": 1,
                "max": 10
            },
            "learning_rate": {
                "values":[1e-4, 1.5e-4, 2e-4]
            },
            "max_lengths": {
                "values": [64, 128, 256]
            }
        },
        "early_terminate": {"type": "hyperband", "min_iter": 3},
    }

    sweep_id = wandb.sweep(config, project="DeBERTa Hyperparameter Tuning")

DeBERTa Implementation and Training

In [33]:
model = DebertaForSequenceClassification.from_pretrained("microsoft/deberta-base")

#180 M
print(f"Model size: {model.num_parameters()}")

Downloading:   0%|          | 0.00/533M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/deberta-base were not used when initializing DebertaForSequenceClassification: ['lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.LayerNorm.weight']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.bias', 'pooler.den

Model size: 139193858


In [34]:
class Trainer_patronise(Trainer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.losses = []

    def compute_loss(self, model, inputs):
        labels = inputs.pop('label')

        outputs = model(**inputs)
        logits = outputs.get('logits')

        loss_task = nn.CrossEntropyLoss()

        loss = loss_task(logits, labels)
        self.losses.append(loss.item())
        
        return loss

In [35]:
def main_patronise():
    if hyperparameter_tuning:
        wandb.init()
        learning_rate = wandb.config.learning_rate
        max_length = wandb.config.max_length
        num_epochs = wandb.config.num_epochs
    else:
        learning_rate = 0.0001
        max_length = 128
        num_epochs = 4

    train_dataset = PatroDataset(tokenizer, trainset,
                                 max_length=max_length)

    #call our custom BERT model and pass as parameter the name of an available pretrained model
    model = DebertaForSequenceClassification.from_pretrained("microsoft/deberta-base")
    
    training_args = TrainingArguments(
        output_dir='./',
        learning_rate=learning_rate,
        lr_scheduler_type='linear',
        logging_steps=100,
        per_device_train_batch_size=32,
        num_train_epochs=num_epochs,
        adam_beta1 = 0.9,
        adam_beta2 = 0.999,
        adam_epsilon = 1e-6,
        max_grad_norm = 1.0,
        save_steps = 2500
    )
    trainer = Trainer_patronise(
        model=model,              
        args=training_args,                
        train_dataset=train_dataset,          
        data_collator=train_dataset.collate_fn
    )
    
    trainer.train()
    trainer.save_model('./')

    report, _ = evaluate(model, tokenizer, test_loader, test_df)
    f1_score = report[1]["f1-score"]

    if hyperparameter_tuning:
        wandb.log({"f1_score": f1_score})  
        wandb.join()

    return trainer.losses

In [36]:
if hyperparameter_tuning:
    losses = wandb.agent(sweep_id, function=main_patronise, count=10)
else:
    losses = main_patronise()

Some weights of the model checkpoint at microsoft/deberta-base were not used when initializing DebertaForSequenceClassification: ['lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.LayerNorm.weight']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.bias', 'pooler.den

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


KeyboardInterrupt: ignored

# EVALUATION

In [ ]:
plt.plot(losses)

In [ ]:
def predict_patronise(input, tokenizer, model): 
  model.eval()
  encodings = tokenizer(input, return_tensors='pt', padding=True, truncation=True, max_length=128)
  
  output = model(**encodings)
  preds = torch.max(output.logits, 1)

  return {'prediction':preds[1], 'confidence':preds[0]}

In [ ]:
def evaluate(model, tokenizer, data_loader, test_df):

  total_count = 0
  correct_count = 0 

  preds = []
  tot_labels = []

  test_df['prediction'] = np.nan

  with torch.no_grad():
    for data in tqdm(data_loader):
      labels = {}
      labels['label'] = data['label']

      sentences = data['text']

      pred = predict_patronise(sentences, tokenizer, model)

      preds.append(pred['prediction'].tolist())
      tot_labels.append(labels['label'].tolist())

      for id, data_prediction in zip(data['parid'], pred['prediction']):
        rows = test_df.loc[test_df.par_id == id]
        rows.prediction = data_prediction.tolist()
        test_df.update(rows)

  # with the saved predictions and labels we can compute accuracy, precision, recall and f1-score
  report = classification_report(tot_labels, preds, target_names=[0,1], output_dict= True)

  return report, test_df

In [ ]:
tokenizer = DebertaTokenizer.from_pretrained("microsoft/deberta-base")

#your saved model name here
model_name = './'
model = DebertaForSequenceClassification.from_pretrained(model_name)

# we don't batch our test set unless it's too big
test_loader = DataLoader(test_dataset)

report, annotated_test_dataset = evaluate(model, tokenizer, test_loader, test_df)

print(report)

print(report['accuracy'])
print(report[0]['f1-score'])
print(report[1]['f1-score'])

In [ ]:
annotated_test_dataset[annotated_test_dataset.prediction==1]

In [ ]:
def analysis_on_intensities(annotated_dataset):
    scores = []
    for intensity_value in range(0,5):
        subset_dataset = annotated_dataset.loc[annotated_dataset.intensity.map(int) == intensity_value]
        labels = subset_dataset['label'].tolist()
        preds = subset_dataset['prediction'].tolist()
        score = f1_score(labels, preds)
        print(score)
        scores.append(score)
    return scores

In [ ]:
analysis_on_intensities(annotated_test_dataset)

In [ ]:
def analysis_on_input_size(annotated_dataset):
    for i in range(0,6):
        subset_dataset = annotated_dataset.loc[i*50 <= annotated_dataset.length.map(int) < (i+1)*50]
        


    plt.title("Average Character Length of a sample")
    plt.ylabel("Frequency")
    plt.xlabel("Character Length")

    # series = df["text"].str.len()

    # print(series.max())
    # series.hist(bins=200, figsize=(8, 6))

    # unpatronising_df["text"].str.len().hist(bins=200, alpha=0.5, figsize=(8, 6))
    # patronising_df["text"].str.len().hist(bins=200, alpha=0.5, figsize=(8, 6))

    (unpatcounts, unpatbins) = np.histogram(unpatronising_df["text"].str.len(), bins=100, range=(0, 2000))
    (patcounts, patbins) = np.histogram(patronising_df["text"].str.len(), bins=100, range=(0, 2000))

    factor = 1/len(unpatronising_df["text"])
    plt.hist(unpatbins[:-1], unpatbins, weights=factor*unpatcounts, alpha=0.6)

    factor = 1/len(patronising_df["text"])
    plt.hist(patbins[:-1], patbins, weights=factor*patcounts, alpha=0.6)

    plt.xlim((0, 2000))

In [ ]:
print(report['accuracy'])
print(report[0]['f1-score'])
print(report[1]['f1-score'])

In [ ]:

report = classification_report(tot_labels, preds, target_names=[0,1], output_dict= True)
